In [4]:
from colory.color import Color
import duckdb as ddb

data = ddb.read_parquet("../week3/rplace.parquet")

In [ ]:
# find the user with the highest avg distance between pixel placements (min 100 placements)
avg_distances = ddb.sql(f"""
    WITH min_100 AS (
        SELECT
            user_id_numeric,
        FROM data
        GROUP BY user_id_numeric
        HAVING COUNT(*) >= 100  
    ), user_coords AS (
        SELECT
            d.user_id_numeric as user_id,
            x,
            y,
            LAG(x) OVER (PARTITION BY user_id ORDER BY timestamp) AS prev_x,
            LAG(y) OVER (PARTITION BY user_id ORDER BY timestamp) AS prev_y,
            timestamp
        FROM data AS d
        JOIN min_100 AS m ON m.user_id_numeric = d.user_id_numeric
        ORDER BY user_id, timestamp
    ), distances AS (
        SELECT
            user_id,
            ROUND(SQRT(POWER(x - prev_x, 2) + POWER(y - prev_y, 2)), 2) as eucl_dist
        FROM user_coords
        WHERE prev_x IS NOT NULL AND prev_y IS NOT NULL
    )
    SELECT
        user_id,
        AVG(eucl_dist) as avg_dist
    FROM distances
    GROUP BY user_id
    ORDER BY avg_dist DESC
""").fetchdf()

# print(avg_distances.head(10))

In [6]:
# get the top 10 users with the most unique colors placed
# pixel_data = ddb.sql(f"""
#     SELECT 
#         user_id_numeric, 
#         COUNT(DISTINCT pixel_color) AS color_count
#     FROM data 
#     GROUP BY user_id_numeric
#     ORDER BY color_count DESC
#     LIMIT 10
# """).fetchdf()

# print(pixel_data)

## Ideas
1. Most colorful user
- In duckDB this query works: realized the color pallet was very small and the max amount of unique colors possible to color was very low, so this is not interesting.
2. User with the most distance placed between pixels on average
- 
3. 1 time user statistics vs 2-25 vs 26-100 
- 